In [1]:
import struct
import matplotlib.pyplot as plt
import numpy as np
import os
from scipy import signal
from math import pi

import pywt
import struct

%pylab inline

plt.rc('font',family='Times New Roman',size=10)
markers_freq = ['.','o','v','1','2','3','4','^','<','>','s','p','*','h','H','+','x','D','d','|','_',','] 
fig = plt.figure(figsize=(5,5))

sgn = 1.0
interval = 5
dt = 0.0000001 * interval
fs = 10000000 / interval
start = 250000
end = 350000

result = []
pwd = os.getcwd()

fa = np.arange(20000, 400000 + 1, 10000)

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.5/dist-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['pi']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


<Figure size 360x360 with 0 Axes>

In [2]:
dict_fitting = {}
for f in fa:
    dict_fitting[f] = []
    dict_fitting[f].append([])
    dict_fitting[f].append([])
    dict_fitting[f].append([0])

for dd in np.arange(90,101,10):
    count = 0
    record = []
    while True:
        filepath = pwd + "//" + "input_data" + "//" + "train" + "//" + str(dd) + "//" + str(count)

        if os.path.isfile(filepath) == False:
            break
        
        with open(filepath, "rb") as fb:
            data = fb.read()

        ch1ch2 = struct.unpack("<"+str(int(len(data)/2))+"H", data)
        ch1ch2 = np.array(ch1ch2)
        ch1ch2 = (ch1ch2-8192)*2.5/8192

        datay1 = ch1ch2[::2]
        datay2 = ch1ch2[1::2]
        
        data1 = datay1[start:end:interval]
        data2 = datay2[start:end:interval]

        wavelet = 'morl'
        c = pywt.central_frequency(wavelet)
        scales = np.array(float(c)) * fs / np.array(fa)

        [cfs1,frequencies1] = pywt.cwt(data1,scales,wavelet,dt)
        [cfs2,frequencies2] = pywt.cwt(data2,scales,wavelet,dt)
        power1 = (abs(cfs1)) ** 2
        power2 = (abs(cfs2)) ** 2
        
        flag = 0
        for i,f in enumerate(fa):
            mean1 = power1[i].mean()
            power1[i] = power1[i] / mean1
            mean2 =  power2[i].mean()
            power2[i] = power2[i] / mean2
            temp = signal.correlate(power1[i],power2[i], mode='same',method='fft')
            corr = (np.where(temp == max(temp))[0][0]-len(temp) / 2 ) * dt * 1000
            if dd != 100 and corr == 0:
                dict_fitting[f][2] = 1
            dict_fitting[f][0].append(dd)
            dict_fitting[f][1].append(corr)

        print(filepath)

        count = count + 1
        if count > 10:
            break

/usr/local/lib/python3.5/dist-packages/scipy/signal/signaltools.py:491: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return x[reverse].conj()


/home/adoge/AE-location//input_data//train//90//0
/home/adoge/AE-location//input_data//train//90//1
/home/adoge/AE-location//input_data//train//90//2
/home/adoge/AE-location//input_data//train//90//3
/home/adoge/AE-location//input_data//train//90//4
/home/adoge/AE-location//input_data//train//90//5
/home/adoge/AE-location//input_data//train//90//6
/home/adoge/AE-location//input_data//train//90//7
/home/adoge/AE-location//input_data//train//90//8
/home/adoge/AE-location//input_data//train//90//9
/home/adoge/AE-location//input_data//train//90//10
/home/adoge/AE-location//input_data//train//100//0
/home/adoge/AE-location//input_data//train//100//1
/home/adoge/AE-location//input_data//train//100//2
/home/adoge/AE-location//input_data//train//100//3
/home/adoge/AE-location//input_data//train//100//4
/home/adoge/AE-location//input_data//train//100//5
/home/adoge/AE-location//input_data//train//100//6
/home/adoge/AE-location//input_data//train//100//7
/home/adoge/AE-location//input_data//trai

In [5]:
result = {}
for f in dict_fitting.keys():
    x = np.array(dict_fitting[f][0])
    y = np.array(dict_fitting[f][1])
    func = np.polyfit(x,y,1)
    yf = x * func[0] + func[1]
    error = np.mean((yf - y) ** 2)
    #if error > 0 and dict_fitting[f][2] != 1:
    result[f] = error

In [6]:
min(result.items(), key=lambda x: x[1])

(360000, 2.43780991735537e-05)

In [7]:
result

{20000: 0.009480311983471072,
 30000: 0.0012165413223140494,
 40000: 0.004328111570247933,
 50000: 0.0064198491735537174,
 60000: 0.011832086776859502,
 70000: 0.002788892561983471,
 80000: 0.007715078512396695,
 90000: 0.005639479338842975,
 100000: 0.003067117768595041,
 110000: 0.003685289256198347,
 120000: 0.0019232747933884298,
 130000: 7.448966942148759e-05,
 140000: 0.00104196694214876,
 150000: 0.0021023512396694216,
 160000: 0.0013256446280991729,
 170000: 0.0035903595041322308,
 180000: 0.003989444214876032,
 190000: 0.0035736694214876033,
 200000: 0.0034017086776859506,
 210000: 0.0007567768595041321,
 220000: 0.00013407438016528928,
 230000: 2.7729338842975205e-05,
 240000: 0.0002084462809917355,
 250000: 0.00028546074380165283,
 260000: 0.0002681776859504133,
 270000: 5.494834710743802e-05,
 280000: 7.453925619834712e-05,
 290000: 0.00010406611570247934,
 300000: 4.274173553719007e-05,
 310000: 3.933471074380165e-05,
 320000: 4.098966942148758e-05,
 330000: 3.400413223140